# ParaView Test
This notebook is useful as an extremely minimal, "hello world"-type example. It creates a synthetic data source and computes an isosurface mesh from it. Then, it creates a ParaView display widget showing the primary render view.

In [ ]:
import paraview
from paraview.simple import *

# create the primary 'Render View'
renV = CreateView('RenderView')
renV.ViewSize = [800, 500]
renV.CenterOfRotation = [0,0,0]               #
renV.CameraFocalPoint = renV.CenterOfRotation #
renV.CameraPosition = [50, 0, 0]              # Initial camera settings
renV.CameraViewUp = [0, 1.0, 0]               #
renV.Background = [0.32, 0.34, 0.43]

# create a new 'Wavelet'
wavelet = Wavelet()

# create a new 'Contour' filter to compute an isosurface from our volume
contour = Contour(Input=wavelet)
contour.ContourBy = ['POINTS', 'RTData']
contour.Isosurfaces = [157.0909652709961]
contour.UpdatePipeline()
contourDisplay = Show(contour, renV)
contourDisplay.Representation = 'Surface'
contourDisplay.AmbientColor = contourDisplay.DiffuseColor = [0.286, 0.851, 0.114]

# import the PVDispaly widget, then instantiate one for renV
from ipyparaview.widgets import PVDisplay
PVDisplay(renV)